In [1]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import datetime
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from datetime import timezone

# 1. EXTRACT

## Hurricane Track Data from NOAA

In [2]:
# Hurricane DataFrame of all Hurricanes that passed through Orlando, FL
hurricane_df = pd.read_csv("databootcamp/Hurricane-ETL-Project/Data/storm_data.csv")
hurricane_df.head()

,STORM NAME & YEAR,DATE RANGE,MAX WIND SPEED,MIN PRESSURE,MAX CATEGORY
0,IRMA 2017,"Aug 30, 2017 to Sep 13, 2017",155,914,H5
1,EMILY 2017,"Jul 30, 2017 to Aug 02, 2017",50,1001,TS
2,MATTHEW 2016,"Sep 28, 2016 to Oct 10, 2016",145,934,H5
3,JULIA 2016,"Sep 13, 2016 to Sep 21, 2016",45,1007,TS
4,DEBBY 2012,"Jun 23, 2012 to Jun 27, 2012",55,990,TS


In [3]:
start_dates = []
unix_start_dates = []
unix_end_dates = []
end_dates = []
unix_compliant = []

months = [("Jan", 1), ("Feb", 2), ("Mar", 3), 
          ("Apr", 4), ("May", 5), ("Jun", 6), 
          ("Jul", 7), ("Aug", 8), ("Sep", 9), 
          ("Oct", 10), ("Nov", 11), ("Dec", 12)]

for date in hurricane_df["DATE RANGE"]:
    start_dates.append(date[0:12])
    end_dates.append(date[16:])

for date in start_dates:
    year = int(date[-4:])
    month = date[0:3]
    count = 0
    for x in months:
        if month == months[count][0]:
            month = months[count][1]
        else:
            count += 1
    day = int(date[4:6])
    dt = datetime.datetime(year, month, day)
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    unix_start_dates.append(timestamp)
    
    if year >= 1970:
        unix_compliant.append("True")
    else:
        unix_compliant.append("False")
        

for date in end_dates:
    year = int(date[-4:])
    month = date[0:3]
    count = 0
    for x in months:
        if month == months[count][0]:
            month = months[count][1]
        else:
            count += 1
    day = int(date[4:6])
    dt = datetime.datetime(year, month, day)
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    unix_end_dates.append(timestamp)
    #print(f"The timestamp for the date {month}-{day}-{year} is: {timestamp}")

In [4]:
# Convert Date Range column to UNIX Timestamps
hurricane_df["START DATE"] = unix_start_dates
hurricane_df["END DATE"] = unix_end_dates
hurricane_df["UNIX COMPLIANT"] = unix_compliant

In [5]:
hurricane_df = hurricane_df[hurricane_df["UNIX COMPLIANT"] == "True"]
hurricane_df = hurricane_df[["STORM NAME & YEAR", "DATE RANGE", "START DATE", "END DATE", "MAX WIND SPEED", "MAX CATEGORY"]]
hurricane_df


,STORM NAME & YEAR,DATE RANGE,START DATE,END DATE,MAX WIND SPEED,MAX CATEGORY
0,IRMA 2017,"Aug 30, 2017 to Sep 13, 2017",1.504051e+09,1.505261e+09,155,H5
1,EMILY 2017,"Jul 30, 2017 to Aug 02, 2017",1.501373e+09,1.501632e+09,50,TS
2,MATTHEW 2016,"Sep 28, 2016 to Oct 10, 2016",1.475021e+09,1.476058e+09,145,H5
3,JULIA 2016,"Sep 13, 2016 to Sep 21, 2016",1.473725e+09,1.474416e+09,45,TS
4,DEBBY 2012,"Jun 23, 2012 to Jun 27, 2012",1.340410e+09,1.340755e+09,55,TS
5,FAY 2008,"Aug 15, 2008 to Aug 28, 2008",1.218758e+09,1.219882e+09,60,TS
6,BARRY 2007,"May 31, 2007 to Jun 05, 2007",1.180570e+09,1.181002e+09,50,TS
7,ERNESTO 2006,"Aug 24, 2006 to Sep 04, 2006",1.156378e+09,1.157328e+09,65,H1
8,TAMMY 2005,"Oct 05, 2005 to Oct 07, 2005",1.128470e+09,1.128643e+09,45,TS
9,JEANNE 2004,"Sep 13, 2004 to Sep 29, 2004",1.095034e+09,1.096416e+09,105,H3


## Weather Data from OpenWeather API

# 2. TRANSFORM

# 3. LOAD